In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class AlexNet(nn.Module):
    def __init__(self, num_classes=2):  # Default output classes set to 2 for binary classification
        super(AlexNet, self).__init__()

        # Convolutional Layers
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0)  # First conv layer
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)  # Second conv layer
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)  # Third conv layer
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)  # Fourth conv layer
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)  # Fifth conv layer

        # Pooling Layer
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)

        # Fully Connected Layers
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)  # Flattened size after conv layers is 256 * 6 * 6
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

        # Dropout layers
        self.dropout = nn.Dropout(p=0.5)

        # Batch normalization layers
        self.batchnorm1 = nn.BatchNorm2d(96)
        self.batchnorm2 = nn.BatchNorm2d(256)

    def forward(self, x):
        # Layer 1: Conv -> BatchNorm -> ReLU -> MaxPool
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = self.maxpool(x)

        # Layer 2: Conv -> BatchNorm -> ReLU -> MaxPool
        x = F.relu(self.batchnorm2(self.conv2(x)))
        x = self.maxpool(x)

        # Layer 3: Conv -> ReLU
        x = F.relu(self.conv3(x))

        # Layer 4: Conv -> ReLU
        x = F.relu(self.conv4(x))

        # Layer 5: Conv -> ReLU -> MaxPool
        x = F.relu(self.conv5(x))
        x = self.maxpool(x)

        # Flatten the output for fully connected layers
        x = x.view(-1, 256 * 6 * 6)

        # Fully Connected Layer 1 -> ReLU -> Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        # Fully Connected Layer 2 -> ReLU -> Dropout
        x = F.relu(self.fc2(x))
        x = self.dropout(x)

        # Fully Connected Layer 3 -> Softmax (for multi-class classification)
        x = self.fc3(x)

        return F.softmax(x, dim=1)

# Initialize the model
model = AlexNet(num_classes=10)  # Change num_classes to match your dataset

# Print the model summary
print(model)


AlexNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9216, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=4096, bias=True)
  (fc3): Linear(in_features=4096, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (batchnorm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
